# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [12]:
%%time

data.head()

CPU times: user 164 µs, sys: 0 ns, total: 164 µs
Wall time: 168 µs


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Качество линейной регрессии не изменится

**Обоснование:** 

$$w = (X^T X)^{-1} X^T y$$

$$
w' = (P^T X^T X P)^{-1} (XP)^T y
$$

$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$    

$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$    

$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$    

$$
a_P = X P P^{-1} (X^T X)^{-1} X^T y
$$    

$$
a_P = X (X^T X)^{-1} X^T y
$$  

$$a = X(X^T X)^{-1} X^T y$$

$$ (X^T X)^{-1} X^T y \neq  P^{-1} (X^T X)^{-1} X^T y $$

$$ w \neq w_P $$

соотношение между  𝑤
  и  𝑤𝑝
 

$$ P^{-1} w = w_P $$

$$a=a_p$$

In [5]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [6]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [15]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print(r2_score(target_valid, predictions))

0.43522757127026546


In [8]:
scores = []
for i in range(100):
    matrix = np.random.normal(size=[features_train.shape[1],features_train.shape[1]])
    features_mm = features_train@matrix
    model.fit(features_mm, target_train)
    scores.append(r2_score(target_valid, predictions) - r2_score(target_valid, model.predict(features_valid@matrix)))
print(pd.Series(scores).max())

1.3729262171580103e-10


## Алгоритм преобразования

**Алгоритм**

Создадим обратимую квадратную матрицу размером количества столбцов признаков

Умножим признаки на эту матрицу

Сохраним обратную матрицу, для восстановления данных

In [413]:
def converter(features):
    det = 0
    while det == 0:
        lock = np.random.normal(size=[features.shape[1],features.shape[1]])
        det = np.linalg.det(lock)
    key = np.linalg.inv(lock)
    return pd.DataFrame(features.values@lock,columns= features.columns), key

def unlock(features,key):
    return pd.DataFrame(np.round(features.values@key,0),columns= features.columns)

**Обоснование**

Предсказания модели равны матрице признаков умноженой на веса +сдвиг.

Предсказания модели, признаки которой умножены на ту же матрицу, на которую умножались признаки при обучении будут равны предсказаниям модели до преобразования

## Проверка алгоритма

In [323]:
features_inv = data.drop('Страховые выплаты', axis=1)
target_inv = data['Страховые выплаты']

In [324]:
features_inv, key = converter(features_inv)

In [325]:
features_train_inv, features_valid_inv, target_train_inv, target_valid_inv = train_test_split(
    features_inv, target_inv, test_size=0.25, random_state=12345)

In [326]:
model = LinearRegression()
model.fit(features_train_inv, target_train_inv)
predictions_inv = model.predict(features_valid_inv)
print(r2_score(target_valid_inv, predictions_inv))

0.435227571270258


In [327]:
print(r2_score(target_valid, predictions) - r2_score(target_valid_inv, predictions_inv))

7.438494264988549e-15


Разница r2_score незначительна

In [328]:
features_recover = unlock(features_inv, key)

In [329]:
features_train_rec, features_valid_rec, target_train_rec, target_valid_rec = train_test_split(
    features_recover, target_inv, test_size=0.25, random_state=12345)

In [330]:
model = LinearRegression()
model.fit(features_train_rec, target_train_rec)
predictions_rec = model.predict(features_valid_rec)
print(r2_score(target_valid_rec, predictions_rec))

0.43522757127026546


In [331]:
print(r2_score(target_valid, predictions) - r2_score(target_valid_rec, predictions_rec))

0.0


Разницы в r2_score по изначальным и восстановленным признакам нет

Так же можно хранить преобразованными и целевые признаки

In [332]:
def converter_full(data):
    lock = np.random.normal(size=[data.shape[1],data.shape[1]])
    key = np.linalg.inv(lock)
    return pd.DataFrame(data.values@lock,columns= data.columns), key

def unlock_2(data,key):
    return pd.DataFrame(np.round(data.values@key,0),columns= data.columns)

In [333]:
data_inv, key_2 = converter_full(data)

In [334]:
data_inv_rec = unlock(data_inv, key_2)

In [335]:
features_rec_2 = data_inv_rec.drop('Страховые выплаты', axis=1)
target_rec_2 = data_inv_rec['Страховые выплаты']

In [336]:
features_train_rec_2, features_valid_rec_2, target_train_rec_2, target_valid_rec_2 = train_test_split(
    features_rec_2, target_rec_2, test_size=0.25, random_state=12345)

In [337]:
model = LinearRegression()
model.fit(features_train_rec_2, target_train_rec_2)
predictions_rec_2 = model.predict(features_valid_rec_2)
print(r2_score(target_valid_rec_2, predictions_rec_2))

0.43522757127026546


In [338]:
print(r2_score(target_valid, predictions) - r2_score(target_valid_rec_2, predictions_rec_2))

0.0


Признаки умножают на обратимую матрицу P - интерпретировать такие данные невозможно, но метрика r2 линейной регрессии таких данных будут совпадать с r2 непреобразованых данных.

Для успешного преобразования необходимо создать обратимую квадратную матрицу размером количества столбцов признаков

Умножим признаки на эту матрицу

Сохраним обратную матрицу, для восстановления данных

Проверка алгоритма показывает , что метрика r2 не меняется как для предсказаний по преобразованным данным, так и по восстановленным